# CIAFR-10  image-classfication project

Implemented by Yiyang Zhang 1800013111

Using **ResNet-50** model

About CIAFR-10 Datasets: 
[Datasets](https://www.cs.toronto.edu/~kriz/cifar.html )                         
[References](https://en.wikipedia.org/wiki/CIFAR-10)

## Import modules

In [1]:
#import required packages#
import pickle
import numpy as np 
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, Add, Input, Flatten, ZeroPadding2D, MaxPooling2D, AveragePooling2D
from keras.models import Model
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


## Load Datasets

In [7]:
# If you have already downloaded the dataset and unpackaged it, make file_local true

file_local = True

#load dataset#
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_data():
    
    #load train data
    X_train = []
    Y_train = []
    for i in range(1,6):
        train_batch = unpickle("data_batch_"+ str(i))
        X_orig = train_batch[b"data"]
        Y_orig = train_batch[b"labels"]
        X_processed = X_orig.reshape((10000,3,32,32)).transpose(0,2,3,1).astype('float32')
        Y_processed = to_categorical(np.array(Y_orig),10)
        X_train.append(X_processed)
        Y_train.append(Y_processed)
    X_train = np.concatenate(X_train)
    Y_train = np.concatenate(Y_train)
    
    #load test data
    test_batch = unpickle("test_batch")
    X_orig = test_batch[b"data"]
    Y_orig = test_batch[b"labels"]
    X_test = X_orig.reshape((10000,3,32,32)).transpose(0,2,3,1).astype('float32')
    Y_test = to_categorical(np.array(Y_orig),10)
    
    return X_train, Y_train, X_test, Y_test
    
if file_local :  
    X_train, Y_train, X_test, Y_test = load_data()

else :
    (X_train, Y_train), (X_test, Y_test) = keras.datasets.cifar10.load_data()
    Y_train = to_categorical(np.array(Y_train),10)
    Y_test = to_categorical(np.array(Y_test),10)
    
    
(M, n_H, n_W, n_C) = X_train.shape
input_shape = (n_H,n_W,n_C)


print ("data loading completed")
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))



data loading completed
number of training examples = 50000
number of test examples = 10000
X_train shape: (50000, 32, 32, 3)
Y_train shape: (50000, 10)
X_test shape: (10000, 32, 32, 3)
Y_test shape: (10000, 10)


## Build ResNet-50 Model

The following figure describes in detail the architecture of this  network
<img src="resnet_kiank.png" style="width:850px;height:150px;">

And firstly We will impletemt convolutional residual block and identity residual block
<img src="idblock3_kiank.png" style="width:650px;height:150px;">                       
<caption><center> Identity block </center></caption>

<img src="convblock_kiank.png" style="width:650px;height:150px;">                       
<caption><center> convolutional block </center></caption>


In [18]:
def id_block(X, f, kernel_channels ,activation = 'relu'):
    
    X_shortcut = X
    
    F1,F2,F3 = kernel_channels

    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation(activation)(X)

    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation(activation)(X)

    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)

    X = Add()([X,X_shortcut])
    X = Activation(activation)(X)
    
    return X


def conv_block(X, f, kernel_channels, strides, activation = 'relu'):
    
    X_shortcut = X
    
    F1,F2,F3 = kernel_channels
    
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (strides,strides), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation(activation)(X)

    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation(activation)(X)

    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)

    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (strides,strides), padding = 'valid')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut )

    X = Add()([X,X_shortcut])
    X = Activation(activation)(X)
    
    return X



def ResNet50(Input_shape = (32, 32, 3), classes = 10):
    
    X_input = Input(Input_shape)
    X = ZeroPadding2D((1, 1))(X_input)
    
    #stage 1
    X = Conv2D(64, (3, 3), strides = (2, 2), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    #stage 2
    
    X = conv_block(X , 3 , kernel_channels = [64, 64, 256], strides = 1)
    X = id_block(X, 3, kernel_channels = [64, 64, 256])
    X = id_block(X, 3, kernel_channels = [64, 64, 256])
    
    #stage 3
    
    X = conv_block(X, 3, kernel_channels = [128, 128, 512], strides = 1)
    X = id_block(X, 3, kernel_channels = [128, 128, 512])
    X = id_block(X, 3, kernel_channels = [128, 128, 512])
    X = id_block(X, 3, kernel_channels = [128, 128, 512])
    
    #stage 4
    
    X = conv_block(X, 3, kernel_channels = [256, 256, 1024], strides = 2)
    X = id_block(X, 3, kernel_channels = [256, 256, 1024])
    X = id_block(X, 3, kernel_channels = [256, 256, 1024])
    X = id_block(X, 3, kernel_channels = [256, 256, 1024])
    X = id_block(X, 3, kernel_channels = [256, 256, 1024])
    X = id_block(X, 3, kernel_channels = [256, 256, 1024])
    
    #stage 5
    
    X = conv_block(X, 3, kernel_channels = [512, 512, 2048], strides = 2)
    X = id_block(X, 3, kernel_channels = [512, 512, 2048])
    X = id_block(X, 3, kernel_channels = [512, 512, 2048])
    
    X = AveragePooling2D(pool_size = (2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(1024, activation='relu')(X)
    X = Dense(classes, activation='softmax')(X)
    
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    
    return model



## Train the model

In [ ]:
model = ResNet50(input_shape, classes= 10)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs = 100, batch_size = 256)

preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

Epoch 1/100
50000/50000 [==============================] - 68s 1ms/step - loss: 1.9772 - accuracy: 0.3443
Epoch 2/100
50000/50000 [==============================] - 53s 1ms/step - loss: 1.1931 - accuracy: 0.5727
Epoch 3/100
50000/50000 [==============================] - 55s 1ms/step - loss: 0.9500 - accuracy: 0.6651
Epoch 4/100
50000/50000 [==============================] - 56s 1ms/step - loss: 0.7792 - accuracy: 0.7263
Epoch 5/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.6482 - accuracy: 0.7716
Epoch 6/100
50000/50000 [==============================] - 57s 1ms/step - loss: 0.5553 - accuracy: 0.8049
Epoch 7/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.4638 - accuracy: 0.8374
Epoch 8/100
50000/50000 [==============================] - 56s 1ms/step - loss: 0.3841 - accuracy: 0.8646
Epoch 9/100
50000/50000 [==============================] - 56s 1ms/step - loss: 0.3250 - accuracy: 0.8863
Epoch 10/100
50000/50000 [====================